In [56]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize, rosen
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
#from sklearn.gaussian_process.kernels import ConstantKernel, WhiteKernel, RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, Sum, Product
from sklearn.gaussian_process.kernels import ConstantKernel as C, WhiteKernel as W, RBF, Matern, RationalQuadratic as RQ, ExpSineSquared as ESS, DotProduct as DP, Sum, Product
import warnings
warnings.filterwarnings("ignore")

#===========================================================================================================#
#Obtaining initial values

def conductivity_func(X, finding_min=False):
    if finding_min==False:
        return (rosen(X.T) * 1e-8).round(6)
    elif finding_min==True:
        return -(rosen(X.T) * 1e-8).round(6)

np.random.seed(10003)
samples = 12
concentrations = 4
X = (np.random.random((samples, concentrations))*10).round(3)
y = conductivity_func(X)

for i in range(len(y)):
    print(f'Sample {i+1}: {X[i].round(3)}, {y[i]}')


#===========================================================================================================#
#Best Kernel

kernel = 31.6**2 * RBF(length_scale=10, length_scale_bounds=(10.0, 10000.0)) + DP(sigma_0=1e+04, sigma_0_bounds=(0.001, 10000.0))
model = GaussianProcessRegressor(kernel = kernel, normalize_y=True, alpha=1e-8, n_restarts_optimizer=3)
'''
score = cross_val_score(model, X, -y, cv=11, scoring='neg_mean_absolute_error')
print(score)
print(np.mean(score))
   ''' 
#===========================================================================================================#
#Cross fold testing
'''
param_grid = [
    {'alpha': [10e-10, 10e-9, 10e3, 10e4, 10e5]}
]

param_search = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=11, n_jobs=-1)

param_search.fit(X, -y)

best_parameters = param_search.best_params_
best_score = param_search.best_score_

print(best_parameters)
print(best_score)
'''
#===========================================================================================================#
#Baysian Optimization

def guess_in_bounds(bounds):
    return [(b[1] - b[0]) * np.random.random() + b[0] for b in bounds]

def acquisition_function1(x, model, explore_weight=1.0):
    pred, std = model.predict([x], return_std=True)
    return -(pred - std * explore_weight)

def acquisition_function2(x, model, explore_weight=1.0):
    pred, std = model.predict([x], return_std=True)
    return -(pred + std * explore_weight)

def acquisition_function3(x, model, best_y, explore_weight=1.0):
    pred, std = model.predict([x], return_std=True)
    return -(norm.cdf((pred - best_y)/std))

bounds = [(0,10)]*concentrations

#===========================================================================================================#
#Global Minimum

model.fit(X, y)

def remove_duplicate(tuple_list):
    for first_index in range(len(tuple_list)-1):
        for second_index in list(range(first_index+1, len(tuple_list))[::-1]):
                #print(np.allclose(global_mins[first_index][0], global_mins[second_index][0], atol=1e-5), (np.isclose(global_mins[first_index][1], global_mins[second_index][1], atol=1e-5)))
                if (np.allclose(tuple_list[first_index][0], tuple_list[second_index][0], atol=1e-4)) and (np.isclose(tuple_list[first_index][1], tuple_list[second_index][1], atol=1e-4)):
                        tuple_list.pop(second_index)
    return tuple_list

def min_over_bounds(func, bounds, args=None, model_used=False):
    global_mins = []
    iterations = 12
    if model_used == True:
        model = args[0]
    for i in range(iterations):
        np.random.seed(i)
        guess = guess_in_bounds(bounds)
        global_min = minimize(func, guess, bounds=bounds, method='Nelder-Mead', args=args)
        if model_used == False:
            global_mins.append((global_min.x, -global_min.fun))
        else:
            global_mins.append((global_min.x, model.predict(global_min.x.reshape(1, -1))))
    remove_duplicate(global_mins)
    print(global_mins) 
    for count, mins in enumerate(global_mins):
        print(count, mins)
       
    return max(global_mins, key=lambda x: x[1])

def ML_func(guess, model):
    return -model.predict(guess.reshape(1, -1))

#rosen_min = min_over_bounds(conductivity_func, bounds, args=(True))
GPR_min = min_over_bounds(ML_func, bounds, model)
#print(rosen_min)
#print(GPR_min)
#print(max(y))

af1_min = min_over_bounds(acquisition_function1, bounds, args=(model, 1.0), model_used=True)
af2_min = min_over_bounds(acquisition_function2, bounds, args=(model, 10.0), model_used=True)
af3_min = min_over_bounds(acquisition_function3, bounds, args=(model, max(y), 10.0), model_used=True)
#print(af1_min[0], conductivity_func(af1_min[0]), model.predict(af1_min[0].reshape(1, -1)))
#print(af2_min[0], conductivity_func(af2_min[0]), model.predict(af2_min[0].reshape(1, -1)))
#print(af3_min[0], conductivity_func(af3_min[0]), model.predict(af3_min[0].reshape(1, -1)))


array1 = (np.array([ 9.99994263,  9.99981511, 10.        , 10.        ]), 0.021216751560901533)
array2 = (np.array([ 9.99994263,  9.99981511, 10.        , 10.        ]), 0.021216751560901533)
print(np.allclose(array1[0], array2[0]))
print(np.allclose(array1[1], array2[1]))

Sample 1: [9.206 5.806 9.795 7.452], 0.014636
Sample 2: [9.123 5.265 8.88  5.74 ], 0.011781
Sample 3: [0.661 2.222 3.34  7.025], 2.3e-05
Sample 4: [2.022 2.641 2.202 2.139], 3.2e-05
Sample 5: [0.501 2.135 3.23  1.475], 8.6e-05
Sample 6: [0.363 1.749 2.184 1.032], 1.7e-05
Sample 7: [5.306 5.682 0.108 5.365], 0.001569
Sample 8: [0.608 8.135 1.662 2.234], 0.004223
Sample 9: [5.049 3.675 4.728 4.297], 0.000879
Sample 10: [7.569 8.265 5.136 5.122], 0.006847
Sample 11: [0.16  6.248 5.637 4.814], 0.001882
Sample 12: [3.649 6.6   2.837 8.771], 0.001704
[(array([10., 10., 10.,  0.]), 0.019905758091267094), (array([10.,  0., 10., 10.]), 0.014653374206405261), (array([ 9.99994263,  9.99981511, 10.        , 10.        ]), 0.021216751560901533), (array([ 0., 10., 10.,  0.]), 0.009688744399667014)]
0 (array([10., 10., 10.,  0.]), 0.019905758091267094)
1 (array([10.,  0., 10., 10.]), 0.014653374206405261)
2 (array([ 9.99994263,  9.99981511, 10.        , 10.        ]), 0.021216751560901533)
3 (array([

kernel = 31.6**2 * RBF(length_scale=10, length_scale_bounds=(10.0, 10000.0)) + DP(sigma_0=1e+04, sigma_0_bounds=(0.001, 10000.0))

model = GaussianProcessRegressor(kernel = kernel, normalize_y=True, alpha=1e-8)

score = -5.251450383038068e-06

error: -0.0012242895406499746

In [ ]:
def min_over_bounds(func, bounds, args, model_used = False):
    global_mins = []
    iterations = 3
    if model_used == True:
        model = args[0]
    for i in range(iterations):
        np.random.seed(i)
        guess = guess_in_bounds(bounds)
        global_min = minimize(func, guess, bounds=bounds, method='Nelder-Mead', args = args)
        if model_used == False:
            global_mins.append((global_min.x, -global_min.fun))
        else:
            global_mins.append((global_min.x, model.predict(global_min.x)))
    return max(global_mins, key=lambda x: x[1])

def min_over_bounds(func, bounds, model = None, acquisition_function_used = False, explore_weight=1.0):
    global_mins = []
    iterations = 3
    if (model == None and acquisition_function_used == False):
        for i in range(iterations):
            np.random.seed(i)
            guess = guess_in_bounds(bounds)
            global_min = minimize(func, guess, bounds=bounds, method='Nelder-Mead', args = True)
            global_mins.append((global_min.x, -global_min.fun))
    elif acquisition_function_used == False:
        for i in range(iterations):
            np.random.seed(i)
            guess = guess_in_bounds(bounds)
            global_min = minimize(func, guess, bounds=bounds, method='Nelder-Mead', args = model)
            global_mins.append((global_min.x, -global_min.fun))
    else:
        for i in range(6):
            np.random.seed(iterations)
            guess = guess_in_bounds(bounds)
            global_min = minimize(func, guess, bounds=bounds, method='Nelder-Mead', args = (model, explore_weight))
            global_mins.append((global_min.x, model.predict(global_min.x)))
    #print(global_mins)
    return max(global_mins, key=lambda x: x[1])

In [63]:
import numpy as np

list_val = [(np.array([10., 10., 10.,  0.]), 0.01990575564384528), #
        (np.array([10., 10., 10.,  0.]), 0.01990575564384528), #
        (np.array([10., 10., 10.,  0.]), 0.01990575564384528), #
        (np.array([10., 10., 10.,  0.]), 0.01990575564384528), 
        (np.array([10., 10., 10., 10.]), 0.021217025951097264), #
        (np.array([10., 10., 10., 10.]), 0.021217025951097264), #
        (np.array([10., 10., 10., 10.]), 0.021217025951097264), 
        (np.array([ 9.99994263,  9.99981511, 10. , 10.]), 0.021216750915081472), 
        (np.array([ 0., 10., 10.,  0.]), 0.009688742594426168), 
        (np.array([ 0., 10., 10.,  0.]), 0.009688742594426168), #
        (np.array([10.,  0., 10., 10.]), 0.014653373453309922), #
        (np.array([10.,  0., 10., 10.]), 0.014653373453309922)]

'''for first_index in range(len(list_val)-1):
        for second_index in list(range(first_index+1, len(list_val))[::-1]):
                print(np.allclose(list_val[first_index][0], list_val[second_index][0], atol=1e-5), (np.isclose(list_val[first_index][1], list_val[second_index][1], atol=1e-5)))
                if (np.allclose(list_val[first_index][0], list_val[second_index][0], atol=1e-4)) and (np.isclose(list_val[first_index][1], list_val[second_index][1], atol=1e-5)):
                        list_val.pop(second_index)


if first_index > len(list_val):
                break

print(len(list_val))
'''

organized = sorted(
    list_val, 
    key=lambda x: x[1],
    reverse=True
)
#print(list_val)
for val in organized:
        print(val)

(array([10., 10., 10., 10.]), 0.021217025951097264)
(array([10., 10., 10., 10.]), 0.021217025951097264)
(array([10., 10., 10., 10.]), 0.021217025951097264)
(array([ 9.99994263,  9.99981511, 10.        , 10.        ]), 0.021216750915081472)
(array([10., 10., 10.,  0.]), 0.01990575564384528)
(array([10., 10., 10.,  0.]), 0.01990575564384528)
(array([10., 10., 10.,  0.]), 0.01990575564384528)
(array([10., 10., 10.,  0.]), 0.01990575564384528)
(array([10.,  0., 10., 10.]), 0.014653373453309922)
(array([10.,  0., 10., 10.]), 0.014653373453309922)
(array([ 0., 10., 10.,  0.]), 0.009688742594426168)
(array([ 0., 10., 10.,  0.]), 0.009688742594426168)
